
欢迎使用“LoRA Model for Radiological Image”，请先从我们的网页获取基础权重文件和您希望识别的任务所对应的任务模型，模型使用的大致步骤如下：
1.下载必要的库函数

In [4]:
!pip install timm==0.9.12
!pip install torch==2.0.1
!pip install torchvision==0.15.2

2.通过timm库以及基础权重导入ViT预训练模型，即您从官网下载的pytorch_model.bin。该模型是12层的在ImageNet数据集上训练过的权重参数量为81.825M，虽然ImageNet不是放射医学数据集，但是这种迁移学习的方式可以让模型更好的学习一般特征提取的方式，显著降低模型对放射医学图像数据量的依赖；
3.通过LoRA.py函数建立LoRA模型，该模型通过在ViT模型的Q、V矩阵外额外添加可训练的模块来实现对原模型的微调；
4.将希望识别的模型权重（对应我们的.safetensor文件，请不要随意修改文件名字，我们的文件是通过文件中的'_'符号读取信息的）导入模型；

In [5]:
from LoRA import LoRA_ViT
import timm
import torch
model = timm.create_model("vit_base_patch16_224",pretrained=True,
                        pretrained_cfg_overlay=dict(file=r"\vit_base_patch16_224\pytorch_model.bin"))
#该步骤导入预训练模型
model2=LoRA_ViT(device=torch.device('cpu'),rank=4)#rank值不影响，会被后续文件覆盖
model2.get_pretrained_weight(model)
model2.load_weights_from_safetensors(r"\ChestXray\ChestXray_base_lora_4_3.safetensors")

5.将希望分析的图片导入，处理后进行分析，并根据我们的表格查找对应的类别，您可以传入多张图片，在jupyterlab中，您运行完前一部分，该部分只需修改文件地址即可在前面的基础上继续运行。

In [15]:
from PIL import Image
from torchvision.transforms import ToTensor,Compose,Resize,Normalize

img_path = r"\train\PNEUMONIA\person23_bacteria_82.jpeg"  # 替换为你的图片路径
img = Image.open(img_path)
image = img.convert('RGB')
# 定义预处理步骤，确保图片适合模型输入
preprocess = Compose([ToTensor(), Resize(size=(224,224)),
                                  Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])  # 标准化
])
# 对图片进行预处理
img_tensor = preprocess(image).unsqueeze(0)
output=model2(img_tensor)
predict = torch.argmax(output, dim=1)
print(predict)

tensor([2])
tensor([[-11.6018,  -8.6222,  13.0603]], grad_fn=<AddmmBackward0>)
